# Spool up EMR Cluster

In [1]:
import boto3
import time
import sys
import botocore
import paramiko
import re
import os
import yaml, re

In [2]:
PATH = os.path.abspath(os.getcwd())
PATH

'/data/hail-on-AWS-spot-instances/src'

In [3]:
#Setup logging
import logging, bluebee
from bluebee import bgp

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
logFile = f'{PATH}/spoolup_EMR_cluster.log'
fh = logging.FileHandler(logFile)
fh.setLevel(logging.INFO)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)-18s-%(levelname)-8s %(message)s', datefmt='%d%b%Y %H:%M:%S')
ch.setFormatter(formatter)
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(ch)
logger.addHandler(fh)
# bgp.api.dump_curl = True
# bluebee.logger.setLevel(logging.DEBUG)

2021021016


In [4]:
#Get the configuration as a yaml object
c=yaml.load(open(PATH+"/config_EMR_spot.yaml"),Loader=yaml.SafeLoader)
logger.info(f'Configuration settings: {c}')

25Mar2021 21:23:55-INFO     Configuration settings: {'config': {'EMR_CLUSTER_NAME': 'tdeboer-hail', 'EC2_NAME_TAG': 'tdeboer-hail-EMR', 'OWNER_TAG': 'tdeboer-ilmn', 'PROJECT_TAG': 'GRE_on_ICA', 'REGION': 'us-east-1', 'MASTER_INSTANCE_TYPE': 'm4.large', 'WORKER_INSTANCE_TYPE': 'r4.4xlarge', 'WORKER_COUNT': '4', 'WORKER_BID_PRICE': '0.50', 'MASTER_HD_SIZE': '250', 'WORKER_HD_SIZE': '500', 'SUBNET_ID': '', 'S3_BUCKET': 's3n://ilmn-hail/', 'KEY_NAME': 'hail-ES-GRE', 'PATH_TO_KEY': '/data/', 'WORKER_SECURITY_GROUP': 'sg-0df1e5704ca2a8196', 'MASTER_SECURITY_GROUP': 'sg-0bab1202c0aa453b3', 'HAIL_VERSION': 'current'}}


In [5]:
# Spot instances and different CORE/MASTER instances
command='aws emr create-cluster --applications Name=Hadoop Name=Spark --tags \'project='+\
c['config']['PROJECT_TAG']+'\' \'Owner='+c['config']['OWNER_TAG']+\
'\' \'Name='+c['config']['EC2_NAME_TAG']+'\' --ec2-attributes \'{"KeyName":"'+\
c['config']['KEY_NAME']+'","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"'+\
c['config']['SUBNET_ID']+'","EmrManagedSlaveSecurityGroup":"'+c['config']['WORKER_SECURITY_GROUP']+\
'","EmrManagedMasterSecurityGroup":"'+c['config']['MASTER_SECURITY_GROUP']+\
'"}\' --service-role EMR_DefaultRole --release-label emr-5.23.0 --log-uri \''+\
c['config']['S3_BUCKET']+'\' --name \''+c['config']['EMR_CLUSTER_NAME']+\
'\' --instance-groups \'[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":'+\
c['config']['MASTER_HD_SIZE']+',"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"MASTER","InstanceType":"'+\
c['config']['MASTER_INSTANCE_TYPE']+'","Name":"Master-Instance"},{"InstanceCount":'+\
c['config']['WORKER_COUNT']+',"BidPrice":"'+c['config']['WORKER_BID_PRICE']+\
'","EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":'+\
c['config']['WORKER_HD_SIZE']+',"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"CORE","InstanceType":"'+\
c['config']['WORKER_INSTANCE_TYPE']+\
'","Name":"Core-Group"}]\' --configurations \'[{"Classification":"spark","Properties":{"maximizeResourceAllocation":"true"}},{"Classification":"yarn-site","Properties":{"yarn.nodemanager.vmem-check-enabled":"false"},"Configurations":[]}]\' --auto-scaling-role EMR_AutoScaling_DefaultRole --ebs-root-volume-size 32 --scale-down-behavior TERMINATE_AT_TASK_COMPLETION --region '+\
c['config']['REGION']
logger.info(f'Executing following command: \n{command}')

25Mar2021 21:23:55-INFO     Executing following command: 
aws emr create-cluster --applications Name=Hadoop Name=Spark --tags 'project=GRE_on_ICA' 'Owner=tdeboer-ilmn' 'Name=tdeboer-hail-EMR' --ec2-attributes '{"KeyName":"hail-ES-GRE","InstanceProfile":"EMR_EC2_DefaultRole","SubnetId":"","EmrManagedSlaveSecurityGroup":"sg-0df1e5704ca2a8196","EmrManagedMasterSecurityGroup":"sg-0bab1202c0aa453b3"}' --service-role EMR_DefaultRole --release-label emr-5.23.0 --log-uri 's3n://ilmn-hail/' --name 'tdeboer-hail' --instance-groups '[{"InstanceCount":1,"EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":250,"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"MASTER","InstanceType":"m4.large","Name":"Master-Instance"},{"InstanceCount":4,"BidPrice":"0.50","EbsConfiguration":{"EbsBlockDeviceConfigs":[{"VolumeSpecification":{"SizeInGB":500,"VolumeType":"gp2"},"VolumesPerInstance":1}]},"InstanceGroupType":"CORE","InstanceType":"r4.4xlarge","Name":"Core-Group"

In [6]:
cluster_id_json=os.popen(command).read()
#My default profile exports TEXT
cluster_id=re.split('\s',cluster_id_json)[1]
# Gives EMR cluster information
client_EMR = boto3.client('emr', region_name=c['config']['REGION'])
logger.warning(f'Created cluster with ID: "{cluster_id}"')

25Mar2021 21:23:56-INFO     Found credentials in shared credentials file: ~/.aws/credentials
25Mar2021 21:23:56-WARNING  Created cluster with ID: "j-2288PJQB6DWLW"


In [7]:
# Cluster state update
status_EMR='STARTING'
tic = time.time()
# Wait until the cluster is created
logger.info('Creating EMR...')

while (status_EMR!='EMPTY'):
    details_EMR=client_EMR.describe_cluster(ClusterId=cluster_id)
    status_EMR=details_EMR.get('Cluster').get('Status').get('State')
    logger.info('Cluster status: '+status_EMR)
    time.sleep(30)
    if (status_EMR=='WAITING'):
        logger.warning('Cluster successfully created! Starting HAIL installation...')
        toc=time.time()-tic
        logger.warning("Total time to provision your cluster: %.2f "%(toc/60)+" minutes")
        break
    if (status_EMR=='TERMINATED_WITH_ERRORS'):
        err = "Cluster un-successfully created. Ending installation..."
        logger.error(err)
        sys.exit(err)

25Mar2021 21:23:56-INFO     Creating EMR...
25Mar2021 21:23:57-INFO     Cluster status: STARTING
25Mar2021 21:24:27-INFO     Cluster status: STARTING
25Mar2021 21:24:57-INFO     Cluster status: STARTING
25Mar2021 21:25:27-INFO     Cluster status: STARTING
25Mar2021 21:25:57-INFO     Cluster status: STARTING
25Mar2021 21:26:27-INFO     Cluster status: STARTING
25Mar2021 21:26:57-INFO     Cluster status: STARTING
25Mar2021 21:27:28-INFO     Cluster status: STARTING
25Mar2021 21:27:58-INFO     Cluster status: STARTING
25Mar2021 21:28:28-INFO     Cluster status: STARTING
25Mar2021 21:28:58-INFO     Cluster status: STARTING
25Mar2021 21:29:28-INFO     Cluster status: STARTING
25Mar2021 21:29:58-INFO     Cluster status: STARTING
25Mar2021 21:30:29-INFO     Cluster status: STARTING
25Mar2021 21:30:59-INFO     Cluster status: WAITING
25Mar2021 21:31:29-WARNING  Cluster successfully created! Starting HAIL installation...
25Mar2021 21:31:29-WARNING  Total time to provision your cluster: 7.54  mi

In [8]:
# Get public DNS from master node
master_dns=details_EMR.get('Cluster').get('MasterPublicDnsName')
master_IP=re.sub("-",".",master_dns.split(".")[0].split("ec2-")[1])
logger.info(f'Master IP address: {master_IP}')

25Mar2021 21:31:29-INFO     Master IP address: 100.26.246.115


In [9]:
#Get ssh connection to master instance
logger.info('Creating secure SSH connection to instance...')
key = paramiko.RSAKey.from_private_key_file(c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem')
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname=master_IP, username="hadoop", pkey=key)
#Run a quick test on the connection
stdin, stdout, stderr = client.exec_command('echo "Hello"')
assert stdout.readline() == 'Hello\n'

25Mar2021 21:31:29-INFO     Creating secure SSH connection to instance...
25Mar2021 21:31:29-INFO     Connected (version 2.0, client OpenSSH_7.4)
25Mar2021 21:31:29-INFO     Authentication (publickey) successful!


In [10]:
# Copy the key into the master
stdin, stdout, stderr = client.exec_command('mkdir -p ~/.ssh/id_rsa')
command='scp -o \'StrictHostKeyChecking no\' -i '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem hadoop@'+\
master_dns+':/home/hadoop/.ssh/id_rsa'
logger.info(f'Copying the keys to the instance with command: \n{command}')
assert os.system(command) == 0

25Mar2021 21:31:29-INFO     Copying the keys to the instance with command: 
scp -o 'StrictHostKeyChecking no' -i /data/hail-ES-GRE.pem /data/hail-ES-GRE.pem hadoop@ec2-100-26-246-115.compute-1.amazonaws.com:/home/hadoop/.ssh/id_rsa


In [11]:
# Copy the hail installation script into the master
command='scp -o \'StrictHostKeyChecking no\' -i '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem '+\
PATH+'/install_hail_and_python36.sh hadoop@'+master_dns+':/home/hadoop'
logger.info(f'Copying the hail installation script to the instance with command: \n{command}')
assert os.system(command) == 0

25Mar2021 21:31:29-INFO     Copying the hail installation script to the instance with command: 
scp -o 'StrictHostKeyChecking no' -i /data/hail-ES-GRE.pem /data/hail-on-AWS-spot-instances/src/install_hail_and_python36.sh hadoop@ec2-100-26-246-115.compute-1.amazonaws.com:/home/hadoop


In [12]:
# Copy the installation script into the master
command='scp -o \'StrictHostKeyChecking no\' -i '+\
c['config']['PATH_TO_KEY']+c['config']['KEY_NAME']+'.pem '+\
PATH+'/install_python36.sh hadoop@'+master_dns+':/home/hadoop'
logger.info(f'Copying the python installation script to the instance with command: \n{command}')
assert os.system(command) == 0

25Mar2021 21:31:30-INFO     Copying the python installation script to the instance with command: 
scp -o 'StrictHostKeyChecking no' -i /data/hail-ES-GRE.pem /data/hail-on-AWS-spot-instances/src/install_python36.sh hadoop@ec2-100-26-246-115.compute-1.amazonaws.com:/home/hadoop


In [13]:
#Install the software on master
VERSION=c['config']['HAIL_VERSION']
logger.info(f'Installing hail version {VERSION} et. al on Master')
logger.warning(f'This is the public JupyterLab link: "http://{master_IP}:8192"')
command='./install_hail_and_python36.sh -v '+ VERSION
logger.info(f'Executing remote command: "{command}"')
stdin, stdout, stderr = client.exec_command('cd /home/hadoop/')
stdin, stdout, stderr = client.exec_command(command)
for line in stdout:
    logger.info(line.rstrip())

25Mar2021 21:31:30-INFO     Installing hail version current et. al on Master
25Mar2021 21:31:30-WARNING  This is the public JupyterLab link: "http://100.26.246.115:8192"
25Mar2021 21:31:30-INFO     Executing remote command: "./install_hail_and_python36.sh -v current"
25Mar2021 21:31:31-INFO     Keys successfully copied to the worker nodes
25Mar2021 21:31:31-INFO     Loaded plugins: priorities, update-motd, upgrade-helper
25Mar2021 21:31:32-INFO     13 packages excluded due to repository priority protections
25Mar2021 21:31:32-INFO     Package python36-3.6.12-1.20.amzn1.x86_64 already installed and latest version
25Mar2021 21:31:32-INFO     Package python36-devel-3.6.12-1.20.amzn1.x86_64 already installed and latest version
25Mar2021 21:31:32-INFO     Resolving Dependencies
25Mar2021 21:31:32-INFO     --> Running transaction check
25Mar2021 21:31:32-INFO     ---> Package python36-setuptools.noarch 0:36.2.7-1.33.amzn1 will be updated
25Mar2021 21:31:32-INFO     ---> Package python36-setu

In [14]:
# close the client connection
logger.info('Closing the client connection')
client.close()

25Mar2021 21:40:07-INFO     Closing the client connection


In [15]:
logger.warning(f'Successfully started ENR cluster "{cluster_id}"')

25Mar2021 21:40:07-WARNING  Successfully started ENR cluster "j-2288PJQB6DWLW"
